In [9]:
from sqlalchemy import create_engine
import random
import pandas as pd
import numpy as np
import names

### Ufuncs

def _clamp(x):
    return max(0, min(x, 255))


def col_gen():
    return "#{0:02x}{1:02x}{2:02x}".format(
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
    )



## Donut chart

* Two tables: data and fill
* In data table it will have two columns: label and value
* In fill table it will have two columns: label and color
* As next step we need to have a drop down menu which will filter the data which has to be shown in the chart and in the backend we will send a list of key value pair where value will be of the same format as it is for a single selection of the drop down menu.


* __Reference:__ https://codesandbox.io/s/6hr9o

In [19]:
### Generate data

label_len = 10
label = [names.get_full_name() for _ in range(label_len)]
value = [random.randint(1, 100) for _ in range(label_len)]
fill = [col_gen() for _ in range(label_len)]
donut_df = pd.DataFrame({'label': label, 'value': value})
label_fill_df = pd.DataFrame({'label': label, 'fill': fill})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
donut_df.to_sql('donut_data_table', disk_engine, if_exists='append')
label_fill_df.to_sql('donut_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM donut_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM donut_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = ((data_df.set_index('label')).join(fill_df.set_index('label'))).reset_index().to_dict('records')
response_payload

[{'label': 'Antonia West', 'value': 96, 'fill': '#d695db'},
 {'label': 'George Williams', 'value': 93, 'fill': '#91671b'},
 {'label': 'Maria Browning', 'value': 28, 'fill': '#cfa2d6'},
 {'label': 'Rick Glass', 'value': 79, 'fill': '#d162fb'},
 {'label': 'Brian Grainger', 'value': 82, 'fill': '#31dc68'},
 {'label': 'Carol Gustaveson', 'value': 31, 'fill': '#cafbbe'},
 {'label': 'Robert Schwendinger', 'value': 42, 'fill': '#355558'},
 {'label': 'Jose Jones', 'value': 16, 'fill': '#a99bee'},
 {'label': 'Anthony Cockrell', 'value': 91, 'fill': '#54a5a3'},
 {'label': 'Kenneth Jones', 'value': 23, 'fill': '#7e1e13'}]